In [1]:
# pip install chromadb sentence-transformers
import chromadb
from chromadb.utils import embedding_functions

sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-mpnet-base-v2", device="cuda"
)

chroma_client = chromadb.PersistentClient(path="/home/ansel/chromadb")

try:
    collection = chroma_client.get_collection(
        name="roderick", embedding_function=sentence_transformer_ef
    )
except:
    collection = chroma_client.create_collection(
        name="roderick", embedding_function=sentence_transformer_ef
    )

/home/ansel/.pyenv/versions/3.12.7/envs/transcribe/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [2]:
import os
from transcriptTools import datesDict, titlesDict

ROOT = os.getcwd()

rotl_titles = titlesDict("rotl_titles.txt")
rotl_dates = datesDict("rotl_dates.txt")
roadwork_titles = titlesDict("roadwork_titles.txt")
roadwork_dates = datesDict("roadwork_dates.txt")

(…)peakerverification_en_titanet_large.nemo:   0%|          | 0.00/102M [00:00<?, ?B/s]

In [3]:
chunkedDir = os.path.join(ROOT, "chunked")
dirs = [(os.path.join(chunkedDir, dir), dir) for dir in sorted(os.listdir(chunkedDir))]

files = [
    (os.path.join(dir, file), showname, file.split("_-_")[0], file)
    for dir, showname in dirs
    for file in sorted(os.listdir(dir))
]

In [4]:
for file in files:
    filepath, showname, episode, filename = file
    chunks = open(filepath, encoding="utf-8").read().split("\n\n")
    podcast = "Roderick on the Line" if showname == "rotl" else "Roadwork"
    hosts = (
        "John Roderick and Merlin Mann"
        if showname == "rotl"
        else "John Roderick and Dan Benjamin"
    )

    documents = chunks
    metadatas = [
        {
            "podcast": podcast,
            "hosts": hosts,
            "episode": episode,
            "title": rotl_titles[episode],
            "date": rotl_dates[episode],
        }
        for idx in enumerate(chunks)
    ]
    ids = [f"{showname}_{episode}_{idx}" for idx, chunk in enumerate(chunks)]
    collection.add(documents=documents, metadatas=metadatas, ids=ids)